In [12]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
import scipy.ndimage as ndimage

In [7]:
capture = cv2.VideoCapture('video.mov')

n = 0
frames = []
while True:
    successful, next_frame = capture.read()
    if not successful:
        # No more frames to read
        print("Processed %d frames" % n)
        break
    frames.append(next_frame)
    n += 1
# Now we have an image! We can process that as we would.

# We have to give up the file at the end.
capture.release()

Processed 101 frames


In [9]:
height, width, channels = frames[0].shape
height, width, channels


(1920, 1440, 3)

In [14]:
def downsize_frame(frame, factor = 2, sigma = 9):
  frame_blurred = cv2.GaussianBlur(frame, (sigma, sigma), 0)
  return frame_blurred[::factor, ::factor]

factor = 4
downsized_size = (width // factor, height // factor) # w,h
output_path = 'output_downsized.mp4'
output_format = cv2.VideoWriter_fourcc('M','P','4','V')
output_fps = 30
downsized_output = cv2.VideoWriter(output_path, output_format, output_fps, downsized_size)

for frame in frames:
  output_frame = downsize_frame(frame, factor=factor)
  downsized_output.write(output_frame)

downsized_output.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [13]:
import os
print(os.getcwd())

/Users/masonfang/Documents/6.8301/project


In [106]:
# first model
# a bunch of 3x3 convolutions
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

import torch.nn as nn
import torch.nn.functional as F

# test = cv2.resize(frames[0], (10, 10))
# test_r = torch.tensor(test[:, :, 0]) # red component
bw_frames = np.array([np.sum(frame, axis=-1)/3 for frame in frames]).astype('float32')

test = torch.tensor(bw_frames[0])
test = test[None,:,:]

print("hi")

Using cpu device
hi


In [107]:
print(test[:,:,0])

tensor([[55.6667, 56.6667, 58.0000,  ..., 29.3333, 29.3333, 29.3333]])


In [113]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 3, 3, bias=False)
        
        # self.conv2 = nn.Conv2d(3, 3, 3)
        # add more later?

    def forward(self, x):
        out = F.relu(self.conv1(x))
        # out = F.relu(self.conv2(out))
        return out
net = Net()
torch.manual_seed(0)
print(test.size())

# a = test.reshape(test, (1, ...))
# print(test_res)


torch.Size([1, 1920, 1440])


In [109]:
print(test.size())
print(test[0])
print(net(test))

torch.Size([1, 1920, 1440])
tensor([[ 55.6667,  56.6667,  58.3333,  ...,  36.3333,  35.3333,  35.3333],
        [ 56.6667,  57.6667,  58.3333,  ...,  35.3333,  35.3333,  35.3333],
        [ 58.0000,  58.0000,  58.3333,  ...,  35.3333,  34.3333,  34.3333],
        ...,
        [ 29.3333,  29.3333,  29.3333,  ..., 126.6667, 139.6667, 145.6667],
        [ 29.3333,  29.3333,  29.3333,  ..., 106.6667, 126.6667, 143.6667],
        [ 29.3333,  28.3333,  28.3333,  ...,  89.6667, 108.6667, 133.6667]])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], grad_fn=<ReluBackward0>)


In [37]:
print(bw_frames[0])
print(frames[0].shape)
print(len(frames))

# print(frames[0])
# in_video = torch.tensor(frames)

[[167 170 175 ... 109 106 106]
 [170 173 175 ... 106 106 106]
 [174 174 175 ... 106 103 103]
 ...
 [ 88  88  88 ... 380 419 437]
 [ 88  88  88 ... 320 380 431]
 [ 88  85  85 ... 269 326 401]]
(1920, 1440, 3)
101
